## Test AttentionDecoder

In [ ]:
from importlib import reload

import numpy as np
import torch
import torch.nn.functional as F
from datasets.tsp import gen_fully_connected_graph
from models.decoder import AttentionDecoder
from torch_geometric.data import Batch, Data
from torch_geometric.nn import global_max_pool
from torch_geometric.utils import to_dense_batch

In [ ]:
embed_dim = 4
num_heads = 2
attn = AttentionDecoder(query_dim=embed_dim, embed_dim=embed_dim, num_heads=num_heads)
attn

In [ ]:
g1 = gen_fully_connected_graph(5)
g2 = gen_fully_connected_graph(10)
g1.mask = torch.ones((51), dtype=torch.bool)
g2.mask = torch.ones((101), dtype=torch.bool)
g1.mask[0:4] = False
g2.mask[0:4] = False
data_list = [g1 if (i % 2) == 0 else g2 for i in range(64)]

In [ ]:
batch = Batch.from_data_list(data_list)
batch

In [ ]:
dense_batch = to_dense_batch(batch.x, batch.batch)[0]
dense_mask = to_dense_batch(batch.mask, batch.batch)[0]

In [ ]:
key = dense_batch.permute(102)
key.shape

In [ ]:
mask = dense_mask.permute(021)
mask.shape

In [ ]:
query = global_max_pool(batch.x, batch.batch)
query = query[None, :, :]
query.shape

In [ ]:
attn_weight = attn(query, key, ~mask)

In [ ]:
mask[0:2]

In [ ]:
attn_weight.exp()[0:2]

In [ ]:
attn_weight.exp().squeeze().multinomial(1)[0:2]

## Test TSP Environment

In [ ]:
import torch
from datasets.tsp import gen_fully_connected_graph
from environments.tsp import TSPEnv
from torch_geometric.data import Batch
from torch_geometric.utils import to_dense_batch

In [ ]:
num_nodes = 10
batch_size = 5

In [ ]:
g = gen_fully_connected_graph(num_nodes)
g_list = [g for _ in range(batch_size)]
batch = Batch.from_data_list(g_list)
batch

In [ ]:
node_pos, dense_mask = to_dense_batch(batch.pos, batch.batch)
assert dense_mask.all()
node_pos.shape

In [ ]:
env = TSPEnv(node_pos)
assert env.num_nodes == num_nodes
assert env.batch_size == batch_size
print(env._avail_mask)

In [ ]:
a = env.random_action()
env.step(a)

In [ ]:
done = False
while not done:
    state, reward, done, _ = env.step(env.random_action())
state, reward

## Test TSPAgent, TSPCritic

In [ ]:
from collections import namedtuple

import torch
from datasets.tsp import gen_complete_graph
from environments.tsp import TSPEnv
from models.tsp_agent import TSPAgent
from models.tsp_baseline import CriticBaseline
from torch_geometric.data import Batch
from torch_geometric.utils import to_dense_batch

In [ ]:
num_nodes = 10
batch_size = 64

class args:
    input_dim = 4
    embed_dim = 64
    num_embed_layers = 2
    num_gnn_layers = 2
    encoder_num_heads = 1
    decoder_num_heads = 1
    bias = True
    pooling_method = "add"
    decode_type = "sampling"
    eval_batch_size = 64
    warmup_batch_size = 256
    device = torch.device("cp")
    max_grad_norm = 1.0
    tanh_clipping = 0
    normalization = "batch"

In [ ]:
model = TSPAgent(args)

In [ ]:
critic = CriticBaseline(args).to(args.device)

In [ ]:
sum(param.numel() for param in critic.parameters())

In [ ]:
batch = Batch.from_data_list([gen_fully_connected_graph(num_nodes) for _ in range(batch_size)])
batch = batch.to(args.device)

In [ ]:
node_pos = to_dense_batch(batch.pos, batch.batch)[0]
env = TSPEnv(node_pos)
env

In [ ]:
model.encode(batch)

In [ ]:
critic(batch)

In [ ]:
log_p_s = []
selected_s = []
reward_s = []
done = False
state = env.reset(node_pos)
step = 0
while (not done) and (step < 999):
    selected, log_p = model(state)
    state, reward, done, _ = env.step(selected)
    log_p_s.append(log_p)
    selected_s.append(selected)
    reward_s.append(reward)
    step += 1

In [ ]:
seqs = torch.stack(selected_s, 1)

In [ ]:
logps = torch.stack(log_p_s, 1)

In [ ]:
log_likelihood = logps.gather(2, seqs).squeeze().sum(1)
log_likelihood

In [ ]:
b = model.encode(batch)

In [ ]:
import torch.nn as nn
ll = logps[:, :, 1:]+0.001
ll
# loss = nn.NLLLoss()(, seqs.squeeze(-1))

In [ ]:
from torch_geometric.nn import BatchNorm
norm = BatchNorm(args.embed_dim).to(args.device)
norm(b.x.to(args.device))

## Test Train

In [ ]:
from args import get_args
from datasets.tsp import TSPDataset
from train import rollout, validate
from rl_algorithms.reinforce import _calc_log_likelihood, clip_grad_norms
from train import warmup_baseline
from tqdm import tqdm

In [ ]:
dataset = TSPDataset(1000, args.graph_size, args=args)

In [ ]:
rollout(model, dataset, env, args)

In [ ]:
validate(model, dataset, env, args)

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(
    [{"params": model.parameters(), "lr": 0.001}] + 
    # [{"params": critic.parameters(), "lr": 0.001}]
)
# warmup_baseline(critic, dataset, env, optimizer, args)

In [ ]:
for i in tqdm(range(10)):
    batch = batch.to(args.device)
    model.train()
    model.set_decode_type("sampling")
    log_p_s = []
    selected_s = []
    reward_s = []
    done = False
    state = env.reset(to_dense_batch(batch.pos, batch.batch)[0])
    embed_data = model.init_embed(batch)
    node_embeddings, graph_feat = model.encoder(embed_data)
    fixed = model.precompute_fixed(node_embeddings, graph_feat)
    step = 0
    while (not done) and (step < 999):
        selected, log_p = model(state, fixed)
        state, reward, done, _ = env.step(selected)
        log_p_s.append(log_p)
        selected_s.append(selected)
        reward_s.append(reward)
        step += 1

    _log_p = torch.stack(log_p_s, 1)
    actions = torch.stack(selected_s, 1)
    log_likelihood = _calc_log_likelihood(_log_p, actions)
    reward = -(reward_s[-1])
    # bl_val, bl_loss = critic.eval(batch, reward)
    rl_loss = (reward*log_likelihood).mean()
    # loss = rl_loss + bl_loss

    optimizer.zero_grad()
    rl_loss.backward()
    grad_norms = clip_grad_norms(optimizer.param_groups, args.max_grad_norm)
    optimizer.step()
    

In [ ]:
print(len(optimizer.param_groups))

In [ ]:
_log_p.exp().gather(2, actions).squeeze(-1)[0:8:2]

In [ ]:
print(_log_p[0].exp())

In [ ]:
# print(reward)
# print(bl_val)


## Test MST

In [ ]:
import networkx.algorithms.tree.mst as mst
from datasets.tsp import gen_fully_connected_graph
from torch_geometric.transforms import distance
from torch_geometric.utils import to_networkx, from_networkx, to_dense_batch
from torch_geometric.data import Batch

In [ ]:
graph = gen_fully_connected_graph(50)
graph = Batch.from_data_list([graph] * 5)
graph

In [ ]:
graph = distance.Distance(cat=False)(graph)
graph

In [ ]:
ng = to_networkx(graph, edge_attrs=["edge_attr"], to_undirected=True)

In [ ]:
ng_mst = mst.minimum_spanning_tree(ng, "edge_attr")

In [ ]:
mst_len = []
for e in ng_mst.edges:
    mst_len.append(ng_mst.edges[e]["edge_attr"])

In [ ]:
sum(mst_len)

In [ ]:
import networkx as nx
nx.draw(ng_mst, pos=graph.pos.numpy())
ng_mst

In [ ]:
sum([edge_value["edge_attr"] for edge_value in ng_mst.edges.values()])
mst_g = from_networkx(ng_mst)
mst_b = Batch()
mst_b.batch = graph.batch
mst_b.edge_index = mst_g.edge_index
mst_b.x = graph.x
to_dense_batch(mst_b.x, mst_b.batch)

## Validateto_data_listto_data_list

In [ ]:
import torch
from args import get_args
from environments.tsp import TSPEnv
from models.tsp_agent import TSPAgent
from train import rollout, validate

In [ ]:
args = get_args([])
args.device = torch.device("cuda")
model = TSPAgent(args).to(args.device)
model_path = "/home/pxh/TSP-experiment/outputs/tsp_50/run_20201217T165809/epoch-21.pt"
dataste_path = "/home/pxh/TSP-experiment/datasets/test/test-50-with-optimal.pk"

val_dataset = torch.load(dataste_path)
load_data = torch.load(model_path)
model.load_state_dict(load_data["model"])
env = TSPEnv()


In [ ]:
out = rollout(model, val_dataset, env, args)

In [ ]:
-out.sum()

In [ ]:
torch.sum(torch.tensor([d.len for d in val_dataset]))

In [ ]:
1672.2463/569.4701

In [ ]:
from torch_geometric.data import Batch
b = Batch.from_data_list([val_dataset[10]]).to(args.device)


In [ ]:
state = env.reset(b.pos.unsqueeze(0))
logp_s = []
done = False
while not done:
    model.eval()
    model.set_decode_type("sampling")
    model.encode(b)
    action, log_p = model(state)
    logp_s.append(log_p)
    state, r, done, _ = env.step(action)
    
-r

In [ ]:
for i in range(49):
#     plt.plot(torch.arange(logp_s[i].size(1)), logp_s[i].exp().detach().cpu().squeeze())
#     plt.show()
#     print(i)
#     print(logp_s[i].exp().detach().cpu().squeeze().sum())

In [ ]:
import matplotlib.pyplot as plt

mst_set = torch.load('/home/pxh/TSP-experiment/datasets/test/mst-10-100-optimal.pk')

x = []
y = []
tsp = []

In [ ]:
for data in mst_set:
    x.append(data.num_nodes)
    y.append((data.concorde_len / data.mst_len).item())
    tsp.append(data.concorde_len.item())

In [ ]:
plt.plot(x,y)

In [ ]:
plt.plot(x, tsp)

## Test Dataset

In [ ]:
import datasets.tsp
from torch_geometric.transforms import Distance, KNNGraph
from torch_geometric.utils import sort_edge_index
from torch_geometric.nn import knn_graph
from torch_geometric.data import Data
import torch.multiprocessing as mp

In [ ]:
%load_ext line_profiler

In [ ]:
lprun -f datasets.tsp.gen_fully_connected_graph -f datasets.tsp.gen_knn_graph -f Data.__init__ datasets.tsp.gen_knn_graph(50,5)

In [ ]:
from importlib import reload
reload(datasets.tsp)

In [ ]:
%lprun -f  datasets.tsp.gen_complete_graph datasets.tsp.gen_complete_graph(50)

In [ ]:
torch.reshape(index.T, [-1])

In [ ]:
idx = torch.stack(edge_attr.chunk(50)).squeeze().topk(k=5, dim=1, largest=False).indices[:, None, :].expand(50,2,5)

In [ ]:
knn = torch.stack(edge_index.chunk(50,dim=1)).gather(dim=2, index=idx)
knn.permute(1,0,2).reshape(2,250)

In [ ]:
torch.stack(edge_attr.chunk(50)).squeeze().topk(k=5, dim=1, largest=False).values.reshape(250,1)

In [ ]:
(knn1.edge_index==knn2.edge_index).all()

## Test 2-opt environment

In [ ]:
import environments.tsp_2opt as env2opt
import torch
import numpy as np
from importlib import reload
reload(tsp2opt)

In [ ]:
env = tsp2opt.TSP2OPTEnv()
node_pos = torch.empty((10, 5, 2)).uniform_(0,1)

In [ ]:
state = env.reset(T=10, node_pos=node_pos)
state.curr_tour

In [ ]:
action = env.random_action()
action

In [ ]:
s, r, d, _ = env.step(action)

In [ ]:
s.curr_tour

In [ ]:
rewards = []
state = env.reset(T=100, node_pos=node_pos)
done = False
while not done:
    action = env.random_action()
    state, reward, done, _ = env.step(action)
    rewards.append(reward)
torch.cat(rewards, dim=1).sum(1)

In [ ]:
state.best_edge_list.shape

## Test 2-opt model

In [10]:
import torch
from datasets.tsp import gen_knn_graph
import models.encoder as encoder
import models.tsp_2opt_agent as tsp2opt
import environments.tsp_2opt as env2opt
from importlib import reload
from torch_geometric.utils import to_undirected, remove_self_loops, contains_self_loops, to_dense_batch
from torch_geometric.data import Batch
from tqdm.notebook import tqdm
reload(tsp2opt)

<module 'models.tsp_2opt_agent' from '/home/pxh/Develope/TSP-experiment/models/tsp_2opt_agent.py'>

In [5]:
num_nodes = 20
batch_size = 64

class args:
    graph_size = 20
    node_dim = 2
    edge_dim = 1
    embed_dim = 64
    num_gnn_layers = 2
    tour_gnn_layers = 2
    encoder_num_heads = 1
    decoder_num_heads = 1
    bias = True
    pooling_method = "mean"
    tour_pooling_method = "add"
    decode_type = "sampling"
    eval_batch_size = 64
    warmup_batch_size = 256
    device = torch.device("cuda")
    max_grad_norm = 1.0
    tanh_clipping = 0
    normalization = "batch"
    graph_type = "knn"

In [11]:
enc = encoder.GNNEncoder(
    args.embed_dim,
    args.num_gnn_layers,
    args.encoder_num_heads,
    args.normalization,
    pooling_method=args.pooling_method,
)
model = tsp2opt.TSP2OPTAgent(args)

enc_optimizer = torch.optim.Adam(
    [{"params": enc.parameters(), "lr": 0.0001}]
)
optimizer = torch.optim.Adam(
    [{"params": model.parameters(), "lr": 0.0001}]
)


In [7]:
batch = Batch.from_data_list([gen_knn_graph(num_nodes, 5) for _ in range(batch_size)])
batch

Batch(batch=[1280], edge_attr=[6400, 1], edge_index=[2, 6400], pos=[1280, 2], x=[1280, 4])

In [ ]:
embed_data = model.init_embed(batch)
embed_data

In [ ]:
node_embedding, _ = model.encoder(embed_data)

In [8]:
env = env2opt.TSP2OPTEnv()
state = env.reset(T=10, node_pos=to_dense_batch(batch.pos, batch.batch)[0])

In [ ]:
selected, log_p = model(state, node_embedding, embed_data.batch)

In [ ]:
node_embedding.device

In [ ]:
curr_len = []
best_len = []
embed_data = model.init_embed(batch)
embed_data.x = embed_data.x.detach()
embed_data.edge_attr = embed_data.edge_attr.detach()
node_embedding, _ = enc(embed_data)
enc_optimizer.zero_grad()
for _ in tqdm(range(50)):
    done = False
    log_p_s = []
    action_s = []
    reward_s = []
    state = env.reset(T=20, node_pos=to_dense_batch(batch.pos, batch.batch)[0])
    # embed_data = model.init_embed(batch)
    # node_embedding, _ = model.encoder(embed_data)
    while not done:
        action, log_p = model(state, node_embedding, embed_data.batch)
        state, reward, done, _ = env.step(action.squeeze())
        log_p_s.append(log_p)
        action_s.append(action)
        reward_s.append(reward)
    logps = torch.cat(log_p_s, dim=0)
    actions = torch.cat(action_s, dim=0)
    log_likelihood = logps.gather(-1, actions).sum(1)
    R = torch.cat(reward_s, dim=0)
    loss = (-log_likelihood*R).mean()

    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()

    curr_len.append(state.curr_tour_len.mean())
    best_len.append(state.best_tour_len.mean())
    print(state.best_tour_len.mean().item())

enc_optimizer.step()

In [ ]:
state.curr_edge_list.device

In [12]:
torch.autograd.set_detect_anomaly(True)
done = False
log_p_s = []
action_s = []
reward_s = []
value_s = []
state = env.reset(T=10, node_pos=to_dense_batch(batch.pos, batch.batch)[0])
embed_data = model.init_embed(batch)
node_embedding, _ = model.encoder(embed_data)
while not done:
    action, log_p, value = model(state, node_embedding, embed_data.batch)
    state, reward, done, _ = env.step(action.squeeze())
    log_p_s.append(log_p)
    action_s.append(action)
    reward_s.append(reward)
    value_s.append(value)

In [13]:
logps = torch.stack(log_p_s, dim=0)
actions = torch.stack(action_s, dim=0)
log_likelihood = logps.gather(-1, actions).squeeze()
R = torch.stack(reward_s, dim=0).expand_as(log_likelihood)
loss = (-log_likelihood*R).mean()

In [34]:
x = torch.ones((512,64,128), requires_grad=True)
y = torch.zeros_like(x, requires_grad=False)
loss = (x - y).mean()
loss.backward()
x[0,0,0].grad

<ipython-input-34-70e951880730>:5: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  x[0,0,0].grad
